<a id='gillespie'></a>
# Ejemplo: El algoritmo de Gillespie y la cinética de Michaelis-Menten

La reacción general propuesta por Leonor Michaelis y Maud Menten es:

$$\require{mhchem}
\require{chemmacros}
\ce{E + S <=>[K_{f}][K_{r}] ES ->[K_{cat}] E + P}$$

Sin embargo puede descomponerse en varias reacciones:

$$\ce{E + S ->[K_{f}] ES} \qquad  (1)$$

$$\ce{ES ->[K_{r}] E + S} \qquad  (2)$$

$$\ce{ES ->[K_{cat}] E + P} \qquad (3)$$

Simulaciones de estas reacciones usando ecuaciones diferenciales muestran
[este comportamiento](https://commons.wikimedia.org/wiki/File:Michaelis_Menten_S_P_E_ES.svg#/media/File:Michaelis_Menten_S_P_E_ES.svg).

## El algoritmo de Gillespie

El [algoritmos de Gillespie](https://en.wikipedia.org/wiki/Gillespie_algorithm)
es una forma de simular reacciones químicas de manera estocástica. Para nuestro
ejemplo realizamos los siguientes pasos:

A cada una de estas reacciones le calculamos su propensión:

$$ a1 = K_{f} * E * S $$

$$ a2 = K_{r} * ES $$

$$ a3 = K_{cat} * ES $$

Propensión a que suceda cualquier reacción:

$$ a0 = a1 + a2 + a3 $$

¿Cuánto tiempo tenemos que esperar hasta que suceda una reacción?

## Distribución exponencial negativa

![](https://docs.tibco.com/pub/business-studio-bpm-edition/3.9.0/doc/html/GUID-7FE5E15C-AFEE-481E-A185-D071AB16427C-display.gif)

![](https://miro.medium.com/max/784/1*geT_1CGq7QrLH7qAlRpYuA.png)

$$f(x) = \lambda e^{-\lambda x}$$

![](https://www.statisticshowto.com/wp-content/uploads/2014/06/360px-Exponential_pdf.svg_.png)

$$ E(X) = \frac{1}{\lambda} \quad \therefore \quad \lambda = \frac{1}{E(X)} $$

In [ ]:
from numpy import random
import matplotlib.pyplot as plt

def michaelis_menten(tf=50,enz=200,sus=500,kf=0.001,kr=0.0001,kcat=0.1,p=0,es=0):
    # Aquí estamos definiendo una función (de ahí el 'def')
    # llamada 'michaelis_menten'.

    '''Algoritmo de Gillespie para la simulación estocástica de la cinética enzimática de Michaelis-Menten.
    
    Parámetros
    ----------
    tf:     Tiempo de simulación.
    enz:    Concentración inicial de enzima.
    sus:    Concentración inicial de sustrato.
    kf:     Constante de formación del complejo enzima-sustrato.
    kr:     Constante de disociación del complejo enzima-sustrato.
    kcat:   Constante de catálisis de la enzima.
    p:      Concentración inicial del producto.
    es:     Concentración inicial del complejo enzima-sustrato.
    ---
    «Es como cuando tú tienes un camello, darle comida, qué camello.»
    ---
    '''

    t = 0
    enzima = [enz] # Aquí creamos una lista llamada 'enzima', por ahora con un único valor que será 'e'.
    sustrato = [sus] # Esta lista y las que siguen son lo mismo pero para cada molécula y cada tiempo.
    complejo = [es]
    producto = [p]
    tiempo = [t]

    while (t < tf):
        a1 = kf*enz*sus
        a2 = kr*es
        a3 = kcat*es
        a0 = a1 + a2 + a3

        t = t + random.exponential(1/a0)
        r = random.random()

        if 0 < r < (a1/a0):
            enz -= 1
            sus -= 1
            es += 1
            enzima.append(enz)
            sustrato.append(sus)
            complejo.append(es)
            producto.append(p)
            tiempo.append(t)

        elif (a1/a0) < r < ((a1+a2)/a0):
            enz += 1
            sus += 1
            es -= 1
            enzima.append(enz)
            sustrato.append(sus)
            complejo.append(es)
            producto.append(p)
            tiempo.append(t)

        else:
            enz += 1
            p += 1
            es -= 1
            enzima.append(enz)
            sustrato.append(sus)
            complejo.append(es)
            producto.append(p)
            tiempo.append(t)

    plt.figure(figsize=(10,5))
    plt.plot(tiempo, enzima, 'b',label='Enzima')
    plt.plot(tiempo, sustrato, 'yellowgreen',label='Sustrato')
    plt.plot(tiempo, complejo, 'cyan',label='Complejo')
    plt.plot(tiempo, producto, 'red',label='Producto')
    plt.legend(loc=9)
    plt.show()

In [ ]:
michaelis_menten(tf=50)

[Análisis exhaustivo del algoritmo de Gillespie](http://be150.caltech.edu/2020/content/lessons/12_stochastic_simulation.html)